
# IR spectroscopy

(sec:ir-tutorial)=

The infrared (IR) absorption process involves the absorption of light with low photon energy (from approx. 800 nm to approx. 1000 $\mu$m) which promotes the excitation of molecular vibrations. Of the entire IR photon energy range, the photons which excite fundamental vibrations of covalent bonds have energies in the range 2.5-25 $\mu$m {cite}`Norman2018, Centrone2015`. The absorbed photon energies (i.e. IR absorption peak positions) correspond to the molecular vibrational freqencies, while the IR intensities are related to the IR linear absorption cross-section (**link to response theory**) {cite}`Norman2018`:
\begin{equation}
\sigma_k(\omega)= \frac{\pi}{3\epsilon_0c}\left(\frac{\partial \mu}{\partial Q_k}\right)^2\frac{\gamma_{k}}{(\omega_{k0}-\omega)^2+\gamma_{k}^2}\,.
\end{equation}
Here, $\mu$ is the molecular dipole moment of the electronic ground state, $Q_k$ is the normal coordinate corresponding to the vibrational mode $k$, $\omega$ is the angular frequency of the absorbed electromagnetic radiation, $\omega_{k0}$ is the anugular frequency difference between the excited vibrational state $|k\rangle$ and the ground state, and $\gamma_{k}$ is the half-width broadening associated with the inverse lifetime of $|k\rangle$. As can be seen from the equation, the IR absorption cross-section depends on the derivative of the dipole moment with respect to the normal coordinate $Q_k$. This means that the only modes which have non-zero IR absorption cross-sections are those associated with a change in the dipole moment {cite}`Siebert2007`.

These molecular vibrations constitute fingerprints of different functional groups present in the system and thus are important for chemical characterization and molecular identification **to be refined, to add examples**.

The first step in an IR spectrum calculation is to determine the normal modes and frequencies. These are obtained from the eigenstates and eigenvalues of the Hessian matrix which collects all the second order derivatives of the energy ($E$) with respect to the nuclear coordinates. For the ground state, the Hessian can be determined [analytically](hessians:label), or numerically based on the analytical gradient. The [dipole moment gradient](dipole_mom_gradient:label), required to compute IR absorption cross-sections, can also be determined analytically. 

**To Add: connection between absorption cross-section and the IR intensity calculated in practice.**.

**To Add: anharmonic corrections**.

In [1]:
# Import section
import veloxchem as vlx
import numpy as np
import sys

In [2]:
# Define the molecule and basis set
molecule_string = """
  O 0 0 0
  H 0 0 1.795239827225189
  H 1.693194615993441 0 -0.599043184453037"""
unit = "au"
basis_set_label = 'sto-3g'

molecule = vlx.Molecule.read_str(molecule_string, units=unit)
basis = vlx.MolecularBasis.read(molecule, basis_set_label)
ostream = vlx.OutputStream(sys.stdout)
ostream.print_block(molecule.get_string()) #TODO: show the molecular structure instead
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004673                         
                           H           0.896000004407        0.000000000000       -0.317000001559                         
                                                                                                                          


In [3]:
# Run the scf calculation
method_settings = {} #{'xcfun':'b3lyp'}
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.update_settings({},method_settings)
scfdrv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [ ]:
# Create the Hessian Driver and use it to compute an analytical SCF Hessian
hessian_settings = {'numerical': 'yes', 'do_four_point':'no'}
cphf_settings = {'use_subspace_solver':'yes'}
hessian_drv = vlx.scfhessiandriver.ScfHessianDriver(scfdrv, scfdrv.comm, ostream)
hessian_drv.update_settings(method_settings, hessian_settings, cphf_settings)
hessian_drv.compute(molecule, basis)

                                                                                                                          
                                                    SCF Hessian Driver                                                    
                                                                                                                          


In [ ]:
hessian_drv.print_hessian(molecule)

The normal modes and frequencies are then determined by diagonalizing the Hessian. First, the transation and rotational modes mut be projected out, leaving 3N-6 (or 3N-5 for a linear molecule) vibrational modes. This is done under the hood with the help of geomeTRIC. **add comparison vib-rot removed vs. included?**

The IR intensities are determined as discussed in more details in **add link**

In [6]:
hessian_drv.vibrational_analysis(molecule, basis)

                                                   Vibrational Analysis                                                   
                                                                                                                          
                 Harmonic frequencies (in cm**-1), force constants (in mdyne/A), reduced masses (in amu),                 
                                                IR intensities (in km/mol),                                               
                                         and Cartesian normal mode displacements.                                         
                                                                                                                          
                                                                                                                          
  Index:                        1                              2                              3               
  Frequency:                

In [ ]:
def add_broadening(list_ex_energy, list_osci_strength, line_profile='Lorentzian', line_param=10, step=10):
        x_min = np.amin(list_ex_energy) - 50
        x_max = np.amax(list_ex_energy) + 50
        x = np.arange(x_min, x_max, step)
        y = np.zeros((len(x)))
        #print(x)
        #print(y)

        # go through the frames and calculate the spectrum for each frame
        for xp in range(len(x)):
            for e, f in zip(list_ex_energy, list_osci_strength):
                if line_profile == 'Gaussian':
                    y[xp] += f * np.exp(-(
                        (e - x[xp]) / line_param)**2)
                elif line_profile == 'Lorentzian':
                    y[xp] += 0.5 * line_param * f / (np.pi * (
                        (x[xp] - e)**2 + 0.25 * line_param**2))
        return x, y

In [ ]:
# plot the IR spectrum
from matplotlib import pyplot as plt

plt.figure(figsize=(7,4))
# Spectrum from -CHF 
x1,y1 = hessian_drv.frequencies, hessian_drv.ir_intensities
x1i, y1i = add_broadening(x1,y1,line_profile='Gaussian',line_param=20,step=10)
plt.plot(x1i,y1i)
#plt.plot(x1,y1,'x')
plt.xlabel('wavenumber (cm**-1)')
plt.ylabel('IR intensity (km/mol)')
plt.title("Calculated IR sepctrum of the water molecule")
plt.tight_layout(); plt.show()
# TODO: add comparison to expriment

**To add: visualization of the H2O molecular vibrations**